In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import ccxt

In [7]:
def fetch_historical_data(symbol, timeframe, limit):
    exchange = ccxt.binance()  # Replace with your desired exchange
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [8]:
def preprocess_data(df):
    df['price_change'] = df['close'].pct_change()
    df['target'] = (df['price_change'] > 0).astype(int)
    df.dropna(inplace=True)
    return df

In [9]:
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

In [10]:
def make_predictions(model, X_test):
    return model.predict(X_test)

In [15]:
def get_recommendations(predictions, df):
    latest_data = df.iloc[-1]
    if predictions[-1] == 1:
        return f"Recommendation: Buy {latest_data['close']} {latest_data['symbol']} at {latest_data['close']}"
    else:
        return f"Recommendation: Sell {latest_data['close']} {latest_data['symbol']} at {latest_data['close']}"

In [16]:
def main():
    symbol = 'BTC/USDT'  # Replace with your desired cryptocurrency symbol
    timeframe = '1h'  # Replace with your desired timeframe
    limit = 1000  # Number of historical data points to fetch

    # Fetch historical data
    df = fetch_historical_data(symbol, timeframe, limit)

    # Preprocess data
    df = preprocess_data(df)

    # Select features and target variable
    features = ['open', 'high', 'low', 'close', 'volume']
    X = df[features]
    y = df['target']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    model = train_model(X_train, y_train)

    # Make predictions
    predictions = make_predictions(model, X_test)

    # Provide investment recommendations
    recommendations = get_recommendations(predictions, df)
    print(recommendations)
    
if __name__ == "__main__":
    main()

KeyError: 'symbol'